# 챕터 6: Precognition (단계별로 생각하기)

- [강의](#lesson)
- [연습 문제](#exercises)
- [예시 플레이그라운드](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 강의

누군가가 당신을 깨우고 즉시 여러 복잡한 질문을 던져서 바로 답변해야 한다면, 어떻게 할 것 같나요? 아마도 **먼저 답변을 차근차근 생각할 시간**이 주어졌을 때만큼 잘하지 못할 것입니다.

그런데 뭘까요? Claude도 마찬가지입니다.

**Claude에게 단계별로 생각할 시간을 주는 것은 때로 Claude를 더 정확하게 만듭니다**, 특히 복잡한 작업에서 그렇습니다. 하지만 **생각하는 것은 소리 내어 할 때만 효과가 있습니다**. Claude에게 생각은 하되 답변만 출력하라고 요청할 수는 없습니다 - 이 경우 실제로는 생각이 이뤄지지 않습니다.

### 예시

아래 프롬프트에서, 두 번째 문장이 첫 번째 문장을 반박한다는 것은 인간 독자에게는 명확합니다. 하지만 **Claude는 "관련 없는"이라는 단어를 너무 문자 그대로 받아들입니다**.

In [ ]:
# 프롬프트
PROMPT = """이 영화 리뷰의 감정이 긍정적인가요 부정적인가요?

이 영화는 신선함과 독창성으로 나를 깜짝 놀라게 했습니다. 완전히 관련 없는 소식으로, 나는 1900년부터 바위 밑에서 살고 있었습니다."""

# Claude의 응답 출력
print(get_completion(PROMPT))

Claude의 응답을 개선하기 위해, **Claude가 답변하기 전에 먼저 생각해보도록 허용해봅시다**. 이를 위해 Claude가 작업을 처리하고 생각하기 위해 취해야 하는 단계들을 말 그대로 명시합니다. 역할 프롬프팅과 함께 사용하면, 이는 Claude가 리뷰를 더 깊이 이해할 수 있도록 합니다.

In [ ]:
# 시스템 프롬프트
SYSTEM_PROMPT = "당신은 영화 리뷰를 예리하게 읽는 독자입니다."

# 프롬프트
PROMPT = """이 리뷰의 감정이 긍정적인가요 부정적인가요? 먼저, <positive-argument>와 <negative-argument> XML 태그에 각 측면에 대한 최고의 논증을 작성한 다음 답변하세요.

이 영화는 신선함과 독창성으로 나를 깜짝 놀라게 했습니다. 완전히 관련 없는 소식으로, 나는 1900년부터 바위 밑에서 살고 있었습니다."""

# Claude의 응답 출력
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude는 때로 순서에 민감합니다**. 이 예시는 Claude가 미묘한 텍스트를 이해하는 능력의 경계에 있으며, 이전 예시에서 논증의 순서를 바꿔서 부정이 먼저, 긍정이 두 번째로 오도록 하면, 이는 Claude의 전체 평가를 긍정적으로 바꿉니다.

대부분의 상황에서 (하지만 혼란스럽게도 모든 상황은 아닙니다), **Claude는 두 옵션 중 두 번째를 선택할 가능성이 더 높습니다**, 아마도 웹에서 가져온 훈련 데이터에서 두 번째 옵션이 정답일 가능성이 더 높았기 때문일 것입니다.

In [ ]:
# 프롬프트
PROMPT = """이 리뷰의 감정이 부정적인가요 긍정적인가요? 먼저 <negative-argument>와 <positive-argument> XML 태그에 각 측면에 대한 최고의 논증을 작성한 다음 답변하세요.

이 영화는 신선함과 독창성으로 나를 깜짝 놀라게 했습니다. 관련 없이, 나는 1900년부터 바위 밑에서 살고 있었습니다."""

# Claude의 응답 출력
print(get_completion(PROMPT))

**Claude가 생각하도록 하는 것은 Claude의 답변을 틀린 것에서 맞는 것으로 바꿀 수 있습니다**. Claude가 실수를 하는 많은 경우에 그만큼 간단합니다!

Claude의 답변이 틀린 예시를 살펴보고 Claude에게 생각하도록 요청하는 것이 어떻게 이를 고칠 수 있는지 봅시다.

In [ ]:
# 프롬프트
PROMPT = "1956년에 태어난 배우가 출연한 유명한 영화를 하나 말해주세요."

# Claude의 응답 출력
print(get_completion(PROMPT))

Claude에게 단계별로 생각하도록 요청하여 이를 고쳐봅시다. 이번에는 `<brainstorm>` 태그를 사용하겠습니다.

In [ ]:
# 프롬프트
PROMPT = "1956년에 태어난 배우가 출연한 유명한 영화를 하나 말해주세요. 먼저 <brainstorm> 태그에서 몇몇 배우들과 그들의 출생년도에 대해 브레인스토밍한 다음, 답변을 제시하세요."

# Claude의 응답 출력
print(get_completion(PROMPT))

위의 내용을 변경하지 않고 강의 프롬프트를 실험해보고 싶다면, 강의 노트북의 맨 아래로 스크롤하여 [**예시 플레이그라운드**](#example-playground)를 방문하세요.

---

## 연습 문제
- [연습 문제 6.1 - 이메일 분류](#exercise-61---classifying-emails)
- [연습 문제 6.2 - 이메일 분류 포맷팅](#exercise-62---email-classification-formatting)

### 연습 문제 6.1 - 이메일 분류
이 연습 문제에서는 Claude에게 이메일을 다음 카테고리로 분류하도록 지시할 것입니다:										
- (A) 사전 판매 질문
- (B) 파손 또는 결함 제품
- (C) 청구 질문
- (D) 기타 (설명해 주세요)

연습 문제의 첫 번째 부분에서는, **Claude가 올바른 분류와 오직 분류만 출력하도록** `PROMPT`를 변경하세요. 답변에는 **올바른 선택의 문자 (A - D)와 괄호, 그리고 카테고리 이름이 포함되어야 합니다**.

`EMAILS` 목록의 각 이메일 옆에 있는 주석을 참조하여 해당 이메일이 어떤 카테고리로 분류되어야 하는지 알아보세요.

In [ ]:
# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = """이 이메일을 녹색 또는 파란색으로 분류해 주세요: {email}"""

# Claude의 응답을 위한 프리필 (있는 경우)
PREFILL = ""

# 리스트로 저장된 변수 내용
EMAILS = [
    "안녕하세요 -- 제 Mixmaster4000이 작동할 때 이상한 소리를 냅니다. 또한 타는 전자제품 냄새처럼 약간 연기 나고 플라스틱 냄새가 납니다. 교체가 필요합니다.", # (B) 파손 또는 결함 제품
    "제 Mixmaster 4000을 페인트 섞는 데 사용할 수 있나요, 아니면 음식 섞는 용도로만 사용해야 하나요?", # (A) 사전 판매 질문 또는 (D) 기타 (설명해 주세요)
    "취소한 후 월 요금이 끝나기까지 4개월을 기다렸습니다!! 도대체 무슨 일이 일어나고 있는 건가요???", # (C) 청구 질문
    "어떻게 여기에 왔는지 모르겠어요. 컴퓨터를 잘 못해요. 도와주세요." # (D) 기타 (설명해 주세요)
]

# 이메일당 여러 올바른 분류 가능성을 수용하기 위해 리스트의 리스트로 저장된 올바른 분류
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 정규식 채점을 위해 사용할 각 카테고리의 문자열 값 딕셔너리
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# 이메일 리스트 반복
for i,email in enumerate(EMAILS):
    
    # 이메일 텍스트를 이메일 플레이스홀더 변수에 대입
    formatted_prompt = PROMPT.format(email=email)
   
    # Claude의 응답 받기
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claude의 응답 채점
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Claude의 응답 출력
    print("--------------------------- 변수 대입이 포함된 전체 프롬프트 ---------------------------")
    print("사용자 턴")
    print(formatted_prompt)
    print("\n어시스턴트 턴")
    print(PREFILL)
    print("\n------------------------------------- Claude의 응답 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 채점 ------------------------------------------")
    print("이 연습 문제가 올바르게 해결되었습니다:", grade, "\n\n\n\n\n\n")

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

아직 막혔나요? 예시 솔루션을 보려면 아래 셀을 실행하세요.

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### 연습 문제 6.2 - 이메일 분류 포맷팅
이 연습 문제에서는 위 프롬프트의 출력을 정확히 원하는 방식으로 포맷된 답변을 생성하도록 개선할 것입니다. 

좋아하는 출력 포맷팅 기법을 사용하여 Claude가 올바른 분류의 문자만을 `<answer></answer>` 태그로 감싸도록 하세요. 예를 들어, 첫 번째 이메일에 대한 답변은 정확히 `<answer>B</answer>` 문자열을 포함해야 합니다.

각 이메일에 대해 어떤 문자 카테고리가 올바른지 잊었다면 `EMAILS` 목록의 각 이메일 옆에 있는 주석을 참조하세요.

In [ ]:
# 변수 내용을 위한 플레이스홀더가 있는 프롬프트 템플릿
PROMPT = """이 이메일을 녹색 또는 파란색으로 분류해 주세요: {email}"""

# Claude의 응답을 위한 프리필 (있는 경우)
PREFILL = ""

# 리스트로 저장된 변수 내용
EMAILS = [
    "안녕하세요 -- 제 Mixmaster4000이 작동할 때 이상한 소리를 냅니다. 또한 타는 전자제품 냄새처럼 약간 연기 나고 플라스틱 냄새가 납니다. 교체가 필요합니다.", # (B) 파손 또는 결함 제품
    "제 Mixmaster 4000을 페인트 섞는 데 사용할 수 있나요, 아니면 음식 섞는 용도로만 사용해야 하나요?", # (A) 사전 판매 질문 또는 (D) 기타 (설명해 주세요)
    "취소한 후 월 요금이 끝나기까지 4개월을 기다렸습니다!! 도대체 무슨 일이 일어나고 있는 건가요???", # (C) 청구 질문
    "어떻게 여기에 왔는지 모르겠어요. 컴퓨터를 잘 못해요. 도와주세요." # (D) 기타 (설명해 주세요)
]

# 이메일당 여러 올바른 분류 가능성을 수용하기 위해 리스트의 리스트로 저장된 올바른 분류
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 정규식 채점을 위해 사용할 각 카테고리의 문자열 값 딕셔너리
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# 이메일 리스트 반복
for i,email in enumerate(EMAILS):
    
    # 이메일 텍스트를 이메일 플레이스홀더 변수에 대입
    formatted_prompt = PROMPT.format(email=email)
   
    # Claude의 응답 받기
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claude의 응답 채점
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Claude의 응답 출력
    print("--------------------------- 변수 대입이 포함된 전체 프롬프트 ---------------------------")
    print("사용자 턴")
    print(formatted_prompt)
    print("\n어시스턴트 턴")
    print(PREFILL)
    print("\n------------------------------------- Claude의 응답 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 채점 ------------------------------------------")
    print("이 연습 문제가 올바르게 해결되었습니다:", grade, "\n\n\n\n\n\n")

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### 축하합니다!

여기까지 모든 연습 문제를 해결했다면, 다음 챕터로 넘어갈 준비가 되었습니다. 즐거운 프롬프팅!

---

## 예시 플레이그라운드

이 영역에서는 이 강의에서 보여준 프롬프트 예시들을 자유롭게 실험하고 프롬프트를 조정하여 Claude의 응답이 어떻게 변하는지 확인해볼 수 있습니다.

In [ ]:
# 프롬프트
PROMPT = """이 영화 리뷰의 감정이 긍정적인가요 부정적인가요?

이 영화는 신선함과 독창성으로 나를 깜짝 놀라게 했습니다. 완전히 관련 없는 소식으로, 나는 1900년부터 바위 밑에서 살고 있었습니다."""

# Claude의 응답 출력
print(get_completion(PROMPT))

In [ ]:
# 시스템 프롬프트
SYSTEM_PROMPT = "당신은 영화 리뷰를 예리하게 읽는 독자입니다."

# 프롬프트
PROMPT = """이 리뷰의 감정이 긍정적인가요 부정적인가요? 먼저, <positive-argument>와 <negative-argument> XML 태그에 각 측면에 대한 최고의 논증을 작성한 다음 답변하세요.

이 영화는 신선함과 독창성으로 나를 깜짝 놀라게 했습니다. 완전히 관련 없는 소식으로, 나는 1900년부터 바위 밑에서 살고 있었습니다."""

# Claude의 응답 출력
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# 프롬프트
PROMPT = """이 리뷰의 감정이 부정적인가요 긍정적인가요? 먼저 <negative-argument>와 <positive-argument> XML 태그에 각 측면에 대한 최고의 논증을 작성한 다음 답변하세요.

이 영화는 신선함과 독창성으로 나를 깜짝 놀라게 했습니다. 관련 없이, 나는 1900년부터 바위 밑에서 살고 있었습니다."""

# Claude의 응답 출력
print(get_completion(PROMPT))

In [ ]:
# 프롬프트
PROMPT = "1956년에 태어난 배우가 출연한 유명한 영화를 하나 말해주세요."

# Claude의 응답 출력
print(get_completion(PROMPT))

In [ ]:
# 프롬프트
PROMPT = "1956년에 태어난 배우가 출연한 유명한 영화를 하나 말해주세요. 먼저 <brainstorm> 태그에서 몇몇 배우들과 그들의 출생년도에 대해 브레인스토밍한 다음, 답변을 제시하세요."

# Claude의 응답 출력
print(get_completion(PROMPT))